# Pyomo -  problem ranca

Problem čijim ćemo se rešavanjem baviti u ovoj svesci je takozvani `problem ranca` (engl. knapsack problem) u kojem je cilj iz skupa objekata zadate težine i vrednosti odabrati one čija se težina uklapa u kapacitete ranca ali tako da je ukupna vrednost najveća moguća. Ovaj zadatak se sreće u mnogim varijantama i spada u domen kombinatorne optimizacije. Može se rešavati korišćenjem dinamičkog programiranja, tehnikom *branch-and-bound* i drugima. Mi ćemo ga rešiti produbljivanjem mogućnosti Pyomo biblioteke. 

Varijanta problema koju ćemo posmatrati je takozvani `0-1 problem` u kojem se objekat u rancu može javiti najviše jednom. Naime, ako sa $i$, $i \in \{1, 2, ..., N\}$ obeležimo indekse objekata čije su vrednosti $v_i$ i težine $w_i$, problem koji treba rešiti je $$max \sum_{i=1}^{N}{v_ix_i}$$ uz uslove $ \sum_{i=1}^{N}{w_ix_i}\lt W$ i $x_i \in \{0, 1\}$ za $i \in \{1, 2, ..., N\}$ gde $W$ predstavlja kapacitet ranca.

In [1]:
from pyomo.environ import *

Pretpostavimo da treba spakovati ranac za letnju pauzu. Na raspolaganju su predmeti šešir, naočare, knjiga, peškir i oprema za ronjenje čije su vrednosti redom 2, 8, 3, 4, i 10, a težine redom 1, 1, 2, 3, i 6. Ako je kapacitet ranca 9 kilograma, koje predmete treba izabrati?  <img src='assets/knapsack_problem.png'>

Izdvojićemo informacije navedene uslovima zadatka u posebne liste.

In [2]:
items = ['hat', 'sunglasses', 'book', 'towel', 'snorkeling kit']
values = {'hat':2, 'sunglasses':8, 'book':3, 'towel':4, 'snorkeling kit': 10}
weights = {'hat':1, 'sunglasses':1, 'book':2, 'towel':3, 'snorkeling kit': 6}
max_weight = 9

Potom ćemo kreirati model pozivom funkcije `ConcreteModel`.

In [3]:
model = ConcreteModel()

Pošto u opštem slučaju broj raspoloživih predmeta može da bude jako veliki, nije zahvalno kreirati pojedinačne promenljive za pojedinačne predmete. Zato ćemo koristiti takozvane `indeksirane promenljive` (engl. indexed variables) koje svakoj vrednosti skupa indeksa pridružuju po jednu promenljivu. Blokom koda koji je naveden niže će za svaku vrednost liste predmeta *items* kreirati po jedna promenljiva *x* modela. Pošto promenljive mogu imati samo vrednost 0 ili 1, prilikom kreiranja postavićemo i da je njihov domen vrednost `Binary`.

In [4]:
model.x = Var(items, domain=Binary)

Na primer, promenljivoj koja odgovara indeksu *book* možemo pristupiti na sledeći način:

In [5]:
model.x['book']

Dalje ćemo zapisati ciljnu funkciju. Za formiranje izraza koristićemo Pyomo formu *list comprehension* konstrukcije i sumiranje nad dobijenim vrednostima.

In [6]:
model.value = Objective(expr=sum(values[item]*model.x[item] for item in items), sense=maximize)

Na sličan način zapisaćemo i ograničenje koje se odnosi na težinu objekata u rancu.

In [7]:
model.weight = Constraint(expr=sum(weights[item]*model.x[item] for item in items ) <= max_weight)

Konstruisani model ćemo ispisati funkcijom `pprint`.

In [8]:
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'hat', 'sunglasses', 'book', 'towel', 'snorkeling kit'}

1 Var Declarations
    x : Size=5, Index=x_index
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
                  book :     0 :  None :     1 : False :  True : Binary
                   hat :     0 :  None :     1 : False :  True : Binary
        snorkeling kit :     0 :  None :     1 : False :  True : Binary
            sunglasses :     0 :  None :     1 : False :  True : Binary
                 towel :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    value : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 2*x[hat] + 8*x[sunglasses] + 3*x[book] + 4*x[towel] + 10*x[snorkeling kit]

1 Constraint Declarations
    weight : Size=1, Index=None, Active=True
      

Dalje ćemo rešiti problem korišćenjem GLPK rešavača i ispisati dobijene rezultate.

In [9]:
solver = SolverFactory('glpk')

In [10]:
result = solver.solve(model)

In [11]:
print('Status: ', result.solver.status)

Status:  ok


Vrednosti indeksiranih promenljivih `x` možemo dobiti pozivom funkcije `get_values`, a zatim ih, korišćenjem odgovarajućih indeksa možemo i ispisati. 

In [12]:
x = model.x.get_values()
for item in items: 
    print ('Objekat {0} - {1}'.format(item, x[item]))

Objekat hat - 0.0
Objekat sunglasses - 1.0
Objekat book - 1.0
Objekat towel - 0.0
Objekat snorkeling kit - 1.0


In [13]:
print('Ukupna tezina objekata u rancu: ', model.weight())

Ukupna tezina objekata u rancu:  9.0


In [14]:
print('Ukupna vrednost objekata u rancu: ', model.value())

Ukupna vrednost objekata u rancu:  21.0
